In [1]:
# install Tensorflow
# !pip3 install tensorflow

In [2]:
# automatically restart kernel after installing TF
# import IPython
# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

In [3]:
# load libs
import numpy as np
import pandas as pd
import os
from pathlib import Path 
import tensorflow as tf

print(tf.__version__)

2.13.0


In [4]:
# load data
movies_data = Path(os.path.abspath(os.path.curdir)) / 'movies.csv'
ratings_data = Path(os.path.abspath(os.path.curdir)) / 'ratings.csv'
movies= pd.read_csv(movies_data)
ratings = pd.read_csv(ratings_data)

In [5]:
print(movies.head(),
      ratings.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy      userId  movieId  rating   timestamp
0       1       16     4.0  1217897793
1       1       24     1.5  1217895807
2       1       32     4.0  1217896246
3       1       47     4.0  1217896556
4       1       50     4.0  1217896523


In [6]:
#Merge two datasets
df = pd.merge(ratings, movies, on = 'movieId')
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,16,4.0,1217897793,Casino (1995),Crime|Drama
1,9,16,4.0,842686699,Casino (1995),Crime|Drama
2,12,16,1.5,1144396284,Casino (1995),Crime|Drama
3,24,16,4.0,963468757,Casino (1995),Crime|Drama
4,29,16,3.0,836820223,Casino (1995),Crime|Drama


In [7]:
# check nans, datatype
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105339 entries, 0 to 105338
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     105339 non-null  int64  
 1   movieId    105339 non-null  int64  
 2   rating     105339 non-null  float64
 3   timestamp  105339 non-null  int64  
 4   title      105339 non-null  object 
 5   genres     105339 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 5.6+ MB


In [8]:
# deleting rows with no genres
df = df.drop(df[df.genres=='(no genres listed)'].index)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105332 entries, 0 to 105338
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     105332 non-null  int64  
 1   movieId    105332 non-null  int64  
 2   rating     105332 non-null  float64
 3   timestamp  105332 non-null  int64  
 4   title      105332 non-null  object 
 5   genres     105332 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 5.6+ MB


In [9]:
# top rating movies
df.groupby('title')['rating'].mean().sort_values(ascending = False).head()

title
Star Wreck: In the Pirkinning (2005)       5.0
Chase, The (1994)                          5.0
Animal Farm (1954)                         5.0
Shackleton's Antarctic Adventure (2001)    5.0
Resident Evil: Retribution (2012)          5.0
Name: rating, dtype: float64

In [10]:
# top count (rated) movies
df.groupby('title')['rating'].count().sort_values(ascending = False).head()

title
Pulp Fiction (1994)                 325
Forrest Gump (1994)                 311
Shawshank Redemption, The (1994)    308
Jurassic Park (1993)                294
Silence of the Lambs, The (1991)    290
Name: rating, dtype: int64

In [11]:
# users_movies matrix, users_list and movies_list
users_movies=df.pivot_table('rating','userId','title').fillna(0).sort_index(level=0, ascending=True, axis=0).sort_index(level=0, ascending=True, axis=1)
movies_list=users_movies.columns.to_list()
users_list=users_movies.index.to_list()
users_movies

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 (1979),...,[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),a/k/a Tommy Chong (2005),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
665,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
666,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# check shape
print(df.userId.value_counts(),
df.title.value_counts())

668    5675
575    2837
458    2086
232    1421
310    1287
       ... 
58       20
464      20
288      20
191      20
26       20
Name: userId, Length: 668, dtype: int64 Pulp Fiction (1994)                                      325
Forrest Gump (1994)                                      311
Shawshank Redemption, The (1994)                         308
Jurassic Park (1993)                                     294
Silence of the Lambs, The (1991)                         290
                                                        ... 
Comedy of Terrors, The (1964)                              1
Drunken Angel (Yoidore tenshi) (1948)                      1
Hard Times (1975)                                          1
Woman Is a Woman, A (femme est une femme, Une) (1961)      1
Pawn Sacrifice (2015)                                      1
Name: title, Length: 10316, dtype: int64


In [13]:
# convert user_movies to tf format
users_movies = tf.convert_to_tensor(users_movies)
users_movies

<tf.Tensor: shape=(668, 10316), dtype=float64, numpy=
array([[0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 2.5, ..., 0. , 2. , 3. ]])>

In [14]:
# movies_feats matrix
movies_feats=df[['title','genres']].drop_duplicates('title').sort_values(by='title',ascending=True).set_index('title')
movies_feats

,genres
title,
'71 (2014),Action|Drama|Thriller|War
'Hellboy': The Seeds of Creation (2004),Action|Adventure|Comedy|Documentary|Fantasy
'Round Midnight (1986),Drama|Musical
'Til There Was You (1997),Drama|Romance
"'burbs, The (1989)",Comedy
...,...
loudQUIETloud: A Film About the Pixies (2006),Documentary
xXx (2002),Action|Crime|Thriller
xXx: State of the Union (2005),Action|Crime|Thriller


In [15]:
# formatting genres into the list
movies_feats['genres'] = movies_feats['genres'].str.split('|')
movies_feats

,genres
title,
'71 (2014),"[Action, Drama, Thriller, War]"
'Hellboy': The Seeds of Creation (2004),"[Action, Adventure, Comedy, Documentary, Fantasy]"
'Round Midnight (1986),"[Drama, Musical]"
'Til There Was You (1997),"[Drama, Romance]"
"'burbs, The (1989)",[Comedy]
...,...
loudQUIETloud: A Film About the Pixies (2006),[Documentary]
xXx (2002),"[Action, Crime, Thriller]"
xXx: State of the Union (2005),"[Action, Crime, Thriller]"


In [16]:
# a kind of one-hot encoding for genres for each user
movies_feats = pd.get_dummies(pd.DataFrame(movies_feats['genres'].tolist()).stack()).groupby(level=0).sum()
features_list = movies_feats.columns.to_list()
movies_feats

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0
1,1,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10311,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
10312,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
10313,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
10314,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [17]:
# changind datatype to float (for further operations with tensors)
movies_feats=movies_feats.astype(float)
movies_feats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10316 entries, 0 to 10315
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Action       10316 non-null  float64
 1   Adventure    10316 non-null  float64
 2   Animation    10316 non-null  float64
 3   Children     10316 non-null  float64
 4   Comedy       10316 non-null  float64
 5   Crime        10316 non-null  float64
 6   Documentary  10316 non-null  float64
 7   Drama        10316 non-null  float64
 8   Fantasy      10316 non-null  float64
 9   Film-Noir    10316 non-null  float64
 10  Horror       10316 non-null  float64
 11  IMAX         10316 non-null  float64
 12  Musical      10316 non-null  float64
 13  Mystery      10316 non-null  float64
 14  Romance      10316 non-null  float64
 15  Sci-Fi       10316 non-null  float64
 16  Thriller     10316 non-null  float64
 17  War          10316 non-null  float64
 18  Western      10316 non-null  float64
dtypes: f

In [18]:
# convert movies_feats to tf format
movies_feats = tf.convert_to_tensor(movies_feats)
movies_feats

<tf.Tensor: shape=(10316, 19), dtype=float64, numpy=
array([[1., 0., 0., ..., 1., 1., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]])>

In [19]:
# computing the user_feature (genres) matrix,
# a matrix containing each user's embedding in the 19-dimensional feature space
# the matrix multiplication of the users_movies tensor with the movies_feats tensor
users_feats = tf.matmul(users_movies, movies_feats)
users_feats

<tf.Tensor: shape=(668, 19), dtype=float64, numpy=
array([[1.7600e+02, 1.1450e+02, 8.0000e+00, ..., 1.6650e+02, 4.2000e+01,
        3.5000e+00],
       [3.5000e+01, 4.0000e+01, 9.0000e+00, ..., 4.7000e+01, 0.0000e+00,
        0.0000e+00],
       [4.2000e+01, 3.1000e+01, 7.0000e+00, ..., 7.8000e+01, 1.1000e+01,
        1.2000e+01],
       ...,
       [1.1400e+02, 9.5000e+01, 3.2000e+01, ..., 1.7800e+02, 3.4000e+01,
        1.5000e+01],
       [5.7500e+01, 4.9500e+01, 1.1000e+01, ..., 6.6000e+01, 1.2500e+01,
        0.0000e+00],
       [2.3375e+03, 1.7155e+03, 3.5400e+02, ..., 3.9825e+03, 1.0495e+03,
        5.2150e+02]])>

In [20]:
# normalizing each user feature vector to sum to 1
users_feats = users_feats / tf.reduce_sum(users_feats, axis=1, keepdims=True)
users_feats

<tf.Tensor: shape=(668, 19), dtype=float64, numpy=
array([[0.14654455, 0.09533722, 0.00666112, ..., 0.13863447, 0.03497086,
        0.00291424],
       [0.10769231, 0.12307692, 0.02769231, ..., 0.14461538, 0.        ,
        0.        ],
       [0.0619469 , 0.04572271, 0.01032448, ..., 0.11504425, 0.01622419,
        0.01769912],
       ...,
       [0.06627907, 0.05523256, 0.01860465, ..., 0.10348837, 0.01976744,
        0.00872093],
       [0.07595773, 0.0653897 , 0.01453104, ..., 0.08718626, 0.01651255,
        0.        ],
       [0.06412015, 0.04705802, 0.0097106 , ..., 0.10924427, 0.02878892,
        0.01430531]])>

In [21]:
# ranking feature relevance for each user
top_users_features = tf.nn.top_k(users_feats, len(features_list))[1]
top_users_features

<tf.Tensor: shape=(668, 19), dtype=int32, numpy=
array([[ 0,  7, 16, ..., 12, 18,  6],
       [ 7, 16,  1, ..., 11, 17, 18],
       [ 7,  4, 14, ...,  2,  6,  9],
       ...,
       [ 7,  4, 16, ..., 12,  6, 11],
       [ 7,  4, 14, ...,  6,  9, 18],
       [ 7,  4, 16, ...,  6,  2, 11]])>

In [22]:
# printing genres preference by user
for i in range(len(users_list)):
    feature_names = [features_list[int(index)] for index in top_users_features[i]]
    print('{}: {}'.format(users_list[i], feature_names))

1: ['Action', 'Drama', 'Thriller', 'Crime', 'Adventure', 'Comedy', 'Sci-Fi', 'Romance', 'Mystery', 'War', 'Fantasy', 'Horror', 'IMAX', 'Children', 'Animation', 'Film-Noir', 'Musical', 'Western', 'Documentary']
2: ['Drama', 'Thriller', 'Adventure', 'Comedy', 'Action', 'Romance', 'Sci-Fi', 'Fantasy', 'Children', 'Crime', 'Animation', 'Musical', 'Mystery', 'Documentary', 'Film-Noir', 'Horror', 'IMAX', 'War', 'Western']
3: ['Drama', 'Comedy', 'Romance', 'Thriller', 'Crime', 'Action', 'Adventure', 'Children', 'Fantasy', 'Mystery', 'Western', 'Musical', 'War', 'Sci-Fi', 'Horror', 'IMAX', 'Animation', 'Documentary', 'Film-Noir']
4: ['Drama', 'Comedy', 'Romance', 'War', 'Adventure', 'Crime', 'Thriller', 'Action', 'Mystery', 'Fantasy', 'Children', 'Film-Noir', 'Musical', 'Horror', 'Western', 'Animation', 'Sci-Fi', 'Documentary', 'IMAX']
5: ['Comedy', 'Animation', 'Children', 'Romance', 'Adventure', 'Fantasy', 'Drama', 'Action', 'Musical', 'IMAX', 'Sci-Fi', 'Thriller', 'Crime', 'Horror', 'Myster

40: ['Thriller', 'Comedy', 'Drama', 'Action', 'Romance', 'Mystery', 'Sci-Fi', 'Children', 'Adventure', 'Horror', 'Crime', 'War', 'Fantasy', 'Animation', 'Documentary', 'Film-Noir', 'IMAX', 'Musical', 'Western']
41: ['Comedy', 'Drama', 'Romance', 'Children', 'Crime', 'Animation', 'Fantasy', 'Thriller', 'Adventure', 'Musical', 'Action', 'Sci-Fi', 'Mystery', 'Horror', 'IMAX', 'Documentary', 'Film-Noir', 'War', 'Western']
42: ['Comedy', 'Romance', 'Drama', 'Adventure', 'Action', 'Fantasy', 'Crime', 'Children', 'Animation', 'Thriller', 'Musical', 'Sci-Fi', 'War', 'Documentary', 'IMAX', 'Horror', 'Mystery', 'Film-Noir', 'Western']
43: ['Drama', 'Comedy', 'Adventure', 'Romance', 'Action', 'Fantasy', 'Animation', 'Children', 'Mystery', 'Sci-Fi', 'Thriller', 'War', 'Western', 'Horror', 'Crime', 'Documentary', 'Film-Noir', 'IMAX', 'Musical']
44: ['Drama', 'Comedy', 'Action', 'Thriller', 'Adventure', 'Sci-Fi', 'Crime', 'Fantasy', 'Horror', 'Romance', 'Mystery', 'Animation', 'Children', 'War', 'Mu

79: ['Drama', 'Action', 'Thriller', 'Crime', 'Comedy', 'Adventure', 'Fantasy', 'Romance', 'War', 'Sci-Fi', 'Mystery', 'Horror', 'IMAX', 'Western', 'Children', 'Film-Noir', 'Animation', 'Documentary', 'Musical']
80: ['Drama', 'Comedy', 'Crime', 'Thriller', 'Romance', 'Action', 'Fantasy', 'Mystery', 'War', 'IMAX', 'Children', 'Western', 'Animation', 'Musical', 'Adventure', 'Documentary', 'Film-Noir', 'Horror', 'Sci-Fi']
81: ['Animation', 'Children', 'Adventure', 'Comedy', 'Fantasy', 'Musical', 'Sci-Fi', 'Action', 'Crime', 'Thriller', 'Drama', 'Mystery', 'Romance', 'Horror', 'IMAX', 'Documentary', 'Film-Noir', 'War', 'Western']
82: ['Drama', 'Comedy', 'Romance', 'Crime', 'Thriller', 'Children', 'Musical', 'Adventure', 'Horror', 'Sci-Fi', 'Fantasy', 'Animation', 'Mystery', 'Action', 'Documentary', 'Film-Noir', 'IMAX', 'War', 'Western']
83: ['Action', 'Comedy', 'Adventure', 'Sci-Fi', 'Drama', 'Thriller', 'Romance', 'Children', 'Animation', 'Crime', 'IMAX', 'Fantasy', 'Mystery', 'War', 'Musi

118: ['Action', 'Drama', 'Comedy', 'Romance', 'Thriller', 'Adventure', 'Sci-Fi', 'Crime', 'Fantasy', 'War', 'Mystery', 'Children', 'Western', 'Horror', 'Animation', 'Musical', 'IMAX', 'Documentary', 'Film-Noir']
119: ['Drama', 'Thriller', 'Action', 'Crime', 'War', 'Romance', 'Mystery', 'Comedy', 'Adventure', 'Sci-Fi', 'Horror', 'Western', 'Fantasy', 'Animation', 'Children', 'Documentary', 'Film-Noir', 'IMAX', 'Musical']
120: ['Comedy', 'Musical', 'Drama', 'Romance', 'Children', 'Fantasy', 'Animation', 'Adventure', 'War', 'IMAX', 'Crime', 'Action', 'Sci-Fi', 'Horror', 'Documentary', 'Film-Noir', 'Mystery', 'Thriller', 'Western']
121: ['Comedy', 'Drama', 'Romance', 'War', 'Adventure', 'Sci-Fi', 'Thriller', 'Crime', 'Action', 'Mystery', 'Fantasy', 'Western', 'Film-Noir', 'Animation', 'Musical', 'Children', 'Documentary', 'Horror', 'IMAX']
122: ['Thriller', 'Action', 'Drama', 'Adventure', 'Comedy', 'Crime', 'Sci-Fi', 'Fantasy', 'Mystery', 'Children', 'Animation', 'Romance', 'Horror', 'War'

160: ['Drama', 'Romance', 'Thriller', 'Action', 'Comedy', 'Adventure', 'Crime', 'Sci-Fi', 'Mystery', 'War', 'Fantasy', 'Horror', 'Children', 'Musical', 'Western', 'Animation', 'Film-Noir', 'IMAX', 'Documentary']
161: ['Action', 'Thriller', 'Drama', 'Adventure', 'Sci-Fi', 'Crime', 'Mystery', 'Comedy', 'IMAX', 'Romance', 'Fantasy', 'Children', 'Animation', 'War', 'Horror', 'Western', 'Musical', 'Film-Noir', 'Documentary']
162: ['Drama', 'Thriller', 'Action', 'Adventure', 'Comedy', 'Crime', 'Romance', 'Sci-Fi', 'War', 'IMAX', 'Children', 'Fantasy', 'Horror', 'Mystery', 'Western', 'Animation', 'Documentary', 'Film-Noir', 'Musical']
163: ['Drama', 'Thriller', 'Action', 'Crime', 'Horror', 'Sci-Fi', 'Comedy', 'Adventure', 'War', 'Mystery', 'Romance', 'Fantasy', 'Film-Noir', 'IMAX', 'Animation', 'Musical', 'Western', 'Children', 'Documentary']
164: ['Action', 'Drama', 'Thriller', 'Comedy', 'Crime', 'Adventure', 'Sci-Fi', 'Romance', 'Fantasy', 'Mystery', 'IMAX', 'War', 'Horror', 'Children', 'An

...

464: ['Drama', 'Comedy', 'Romance', 'Western', 'Action', 'Fantasy', 'Thriller', 'Animation', 'Children', 'War', 'Musical', 'Adventure', 'Crime', 'Sci-Fi', 'Horror', 'Mystery', 'Documentary', 'Film-Noir', 'IMAX']
465: ['Comedy', 'Drama', 'Romance', 'Crime', 'Action', 'Adventure', 'Thriller', 'Sci-Fi', 'Fantasy', 'Mystery', 'Animation', 'Children', 'War', 'Film-Noir', 'Horror', 'Documentary', 'Musical', 'IMAX', 'Western']
466: ['Drama', 'Action', 'Thriller', 'Crime', 'Sci-Fi', 'Comedy', 'Adventure', 'Mystery', 'War', 'Romance', 'Fantasy', 'Horror', 'IMAX', 'Animation', 'Western', 'Children', 'Documentary', 'Film-Noir', 'Musical']
467: ['Drama', 'Sci-Fi', 'Thriller', 'Action', 'Romance', 'Adventure', 'IMAX', 'Mystery', 'Comedy', 'Crime', 'Fantasy', 'War', 'Animation', 'Children', 'Documentary', 'Film-Noir', 'Horror', 'Musical', 'Western']
468: ['Drama', 'Comedy', 'Action', 'Adventure', 'Romance', 'Thriller', 'Crime', 'War', 'Horror', 'Western', 'Children', 'Sci-Fi', 'Animation', 'Fantasy'

509: ['Comedy', 'Drama', 'Romance', 'Adventure', 'Action', 'Children', 'Sci-Fi', 'Fantasy', 'Thriller', 'Crime', 'Musical', 'War', 'Animation', 'Western', 'Documentary', 'Film-Noir', 'Horror', 'IMAX', 'Mystery']
510: ['Comedy', 'Drama', 'Adventure', 'Thriller', 'Action', 'Sci-Fi', 'Crime', 'Mystery', 'Horror', 'Romance', 'Fantasy', 'War', 'Children', 'Musical', 'Animation', 'Western', 'Documentary', 'Film-Noir', 'IMAX']
511: ['Comedy', 'Action', 'Drama', 'Adventure', 'Thriller', 'Romance', 'Children', 'Fantasy', 'Sci-Fi', 'Crime', 'War', 'Animation', 'Western', 'Musical', 'IMAX', 'Horror', 'Mystery', 'Documentary', 'Film-Noir']
512: ['Drama', 'Action', 'Thriller', 'Comedy', 'Adventure', 'Crime', 'Mystery', 'Sci-Fi', 'Romance', 'Fantasy', 'War', 'Horror', 'Children', 'Animation', 'Musical', 'IMAX', 'Western', 'Documentary', 'Film-Noir']
513: ['Action', 'Thriller', 'Adventure', 'Sci-Fi', 'Drama', 'Comedy', 'Crime', 'Fantasy', 'Horror', 'Mystery', 'IMAX', 'War', 'Animation', 'Children', '

551: ['Adventure', 'Action', 'Thriller', 'Comedy', 'Drama', 'Children', 'Fantasy', 'Romance', 'Sci-Fi', 'Animation', 'Crime', 'Musical', 'IMAX', 'Mystery', 'War', 'Western', 'Documentary', 'Film-Noir', 'Horror']
552: ['Comedy', 'Adventure', 'Children', 'Fantasy', 'Romance', 'Drama', 'Animation', 'Action', 'Thriller', 'Musical', 'Sci-Fi', 'Crime', 'Documentary', 'Film-Noir', 'Horror', 'IMAX', 'Mystery', 'War', 'Western']
553: ['Comedy', 'Drama', 'Crime', 'Thriller', 'Adventure', 'Fantasy', 'Horror', 'Action', 'Romance', 'Sci-Fi', 'Children', 'War', 'Animation', 'Mystery', 'Documentary', 'Film-Noir', 'IMAX', 'Musical', 'Western']
554: ['Comedy', 'Drama', 'Adventure', 'Action', 'Children', 'Romance', 'Fantasy', 'Sci-Fi', 'Crime', 'Thriller', 'Animation', 'Musical', 'War', 'IMAX', 'Mystery', 'Western', 'Documentary', 'Horror', 'Film-Noir']
555: ['Drama', 'Comedy', 'Thriller', 'Action', 'Romance', 'Adventure', 'Crime', 'Sci-Fi', 'Fantasy', 'Horror', 'Mystery', 'Children', 'Musical', 'Animat

592: ['Drama', 'Thriller', 'Crime', 'Action', 'Comedy', 'Sci-Fi', 'Romance', 'Horror', 'Adventure', 'War', 'Western', 'Musical', 'Fantasy', 'Mystery', 'IMAX', 'Children', 'Documentary', 'Animation', 'Film-Noir']
593: ['Drama', 'Thriller', 'Comedy', 'Crime', 'Mystery', 'Action', 'Adventure', 'Romance', 'War', 'Horror', 'Fantasy', 'Sci-Fi', 'Film-Noir', 'Western', 'Animation', 'Children', 'Musical', 'Documentary', 'IMAX']
594: ['Comedy', 'Adventure', 'Action', 'Romance', 'Thriller', 'Children', 'Drama', 'Fantasy', 'Sci-Fi', 'Crime', 'Musical', 'Animation', 'Horror', 'Mystery', 'Documentary', 'Film-Noir', 'IMAX', 'War', 'Western']
595: ['Action', 'Adventure', 'Comedy', 'Thriller', 'Romance', 'Sci-Fi', 'Drama', 'Fantasy', 'Children', 'Animation', 'Musical', 'Mystery', 'Crime', 'Documentary', 'Film-Noir', 'Horror', 'IMAX', 'War', 'Western']
596: ['Drama', 'Comedy', 'Romance', 'Crime', 'War', 'Action', 'Adventure', 'Thriller', 'Children', 'Film-Noir', 'Mystery', 'Sci-Fi', 'Fantasy', 'Horror'

633: ['Drama', 'Action', 'Adventure', 'Comedy', 'Thriller', 'Romance', 'Sci-Fi', 'Crime', 'Fantasy', 'War', 'Children', 'Mystery', 'Animation', 'Musical', 'Horror', 'IMAX', 'Western', 'Documentary', 'Film-Noir']
634: ['Drama', 'Comedy', 'Mystery', 'Thriller', 'Sci-Fi', 'Romance', 'Adventure', 'Action', 'Fantasy', 'War', 'Horror', 'Crime', 'Children', 'Film-Noir', 'Documentary', 'Musical', 'Western', 'IMAX', 'Animation']
635: ['Thriller', 'Drama', 'Mystery', 'Comedy', 'Action', 'Crime', 'Horror', 'Adventure', 'Fantasy', 'Animation', 'Children', 'Romance', 'War', 'Film-Noir', 'Sci-Fi', 'Documentary', 'IMAX', 'Musical', 'Western']
636: ['Drama', 'Comedy', 'Romance', 'Adventure', 'Crime', 'Thriller', 'Action', 'Sci-Fi', 'Mystery', 'Fantasy', 'Documentary', 'Children', 'War', 'Musical', 'Animation', 'Horror', 'Western', 'Film-Noir', 'IMAX']
637: ['Adventure', 'Action', 'Sci-Fi', 'Comedy', 'Thriller', 'Fantasy', 'Drama', 'Horror', 'Animation', 'Children', 'Crime', 'Romance', 'Musical', 'Myst

In [23]:
# determining movie recommendations (the dot product as the similarity measure)
# the matrix multiplication of the users_feats tensor with the transposed movies_feats tensor
users_ratings = tf.matmul(users_feats, tf.transpose(movies_feats))
users_ratings

<tf.Tensor: shape=(668, 10316), dtype=float64, numpy=
array([[0.4641965 , 0.35220649, 0.14737719, ..., 0.39383847, 0.08742714,
        0.08784346],
       [0.4       , 0.40307692, 0.17230769, ..., 0.29230769, 0.12      ,
        0.14461538],
       [0.40412979, 0.33333333, 0.22713864, ..., 0.24778761, 0.21091445,
        0.20943953],
       ...,
       [0.40290698, 0.2994186 , 0.21686047, ..., 0.25116279, 0.15      ,
        0.14476744],
       [0.37318362, 0.36922061, 0.1988111 , ..., 0.22655218, 0.17569353,
        0.18097754],
       [0.46822109, 0.25728981, 0.2845563 , ..., 0.24869017, 0.12438623,
        0.12856947]])>

In [24]:
# applying a mask to the all_users_ratings matrix to focus only on the ratings for new movies 
# (excluding those already seen before)
users_unseen_movies = tf.equal(users_movies, tf.zeros_like(users_movies))
ignore_matrix = tf.zeros_like(tf.cast(users_movies, tf.float64))

users_ratings_new = tf.where(
    users_unseen_movies,
    users_ratings,
    ignore_matrix)

users_ratings_new

<tf.Tensor: shape=(668, 10316), dtype=float64, numpy=
array([[0.4641965 , 0.35220649, 0.14737719, ..., 0.39383847, 0.08742714,
        0.08784346],
       [0.4       , 0.40307692, 0.17230769, ..., 0.29230769, 0.12      ,
        0.14461538],
       [0.40412979, 0.33333333, 0.22713864, ..., 0.24778761, 0.21091445,
        0.20943953],
       ...,
       [0.40290698, 0.2994186 , 0.21686047, ..., 0.25116279, 0.15      ,
        0.14476744],
       [0.37318362, 0.36922061, 0.1988111 , ..., 0.22655218, 0.17569353,
        0.18097754],
       [0.46822109, 0.25728981, 0.        , ..., 0.24869017, 0.        ,
        0.        ]])>

In [25]:
# printing top 3 rated movies for each user
top=3
top_movies = tf.nn.top_k(users_ratings_new, top)[1]
top_movies

<tf.Tensor: shape=(668, 3), dtype=int32, numpy=
array([[7770, 5617, 5618],
       [8715, 7770, 5617],
       [5617, 5618, 6777],
       ...,
       [7770, 5617, 5618],
       [5617, 5618, 7770],
       [5617, 5618, 4496]])>

In [26]:
# printing recommended movies for each user
for i in range(len(users_list)):
    movie_names = [movies_list[index] for index in top_movies[i]]
    print('{}: {}'.format(users_list[i], movie_names))

1: ['Rubber (2010)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Lupin III: Sweet Lost Night (Rupan Sansei: Sweet Lost Night) (2008)']
2: ['Stunt Man, The (1980)', 'Rubber (2010)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)']
3: ['Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Lupin III: Sweet Lost Night (Rupan Sansei: Sweet Lost Night) (2008)', 'Osmosis Jones (2001)']
4: ['Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Lupin III: Sweet Lost Night (Rupan Sansei: Sweet Lost Night) (2008)', 'Rubber (2010)']
5: ['Gnomeo & Juliet (2011)', 'Ice Age: Dawn of the Dinosaurs (2009)', 'Robots (2005)']
6: ['Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Lupin III: Sweet Lost Night (Rupan Sansei: Sweet Lost Night) (2008)', 'Osmosis Jones (2001)']
7: ['Rubber (2010)', 'Patlabor: The Movie (Kidô keisatsu patorebâ: The Movie) (1989)', 'Pulse (2006)']
8: ['Stunt Man, The (1980)', 'Lupin 

192: ['Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Lupin III: Sweet Lost Night (Rupan Sansei: Sweet Lost Night) (2008)', 'Osmosis Jones (2001)']
193: ['Super Mario Bros. (1993)', 'Aqua Teen Hunger Force Colon Movie Film for Theaters (2007)', 'Mars Needs Moms (2011)']
194: ['Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Lupin III: Sweet Lost Night (Rupan Sansei: Sweet Lost Night) (2008)', 'Osmosis Jones (2001)']
195: ['Rubber (2010)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Lupin III: Sweet Lost Night (Rupan Sansei: Sweet Lost Night) (2008)']
196: ['Stunt Man, The (1980)', 'Rubber (2010)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)']
197: ['Rubber (2010)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Lupin III: Sweet Lost Night (Rupan Sansei: Sweet Lost Night) (2008)']
198: ['Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Lupin III: Sweet L

388: ['Rubber (2010)', 'Blood Diamond (2006)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)']
389: ['Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Lupin III: Sweet Lost Night (Rupan Sansei: Sweet Lost Night) (2008)', 'Osmosis Jones (2001)']
390: ['Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Lupin III: Sweet Lost Night (Rupan Sansei: Sweet Lost Night) (2008)', 'Osmosis Jones (2001)']
391: ['Rubber (2010)', 'Ichi the Killer (Koroshiya 1) (2001)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)']
392: ['Interstate 60 (2002)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Lupin III: Sweet Lost Night (Rupan Sansei: Sweet Lost Night) (2008)']
393: ['Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Lupin III: Sweet Lost Night (Rupan Sansei: Sweet Lost Night) (2008)', 'Rubber (2010)']
394: ['Rubber (2010)', 'Stunt Man, The (1980)', 'Lupin III: First Contact (Rupa

586: ['Rubber (2010)', 'Stunt Man, The (1980)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)']
587: ['Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Lupin III: Sweet Lost Night (Rupan Sansei: Sweet Lost Night) (2008)', 'Rubber (2010)']
588: ['Rubber (2010)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Lupin III: Sweet Lost Night (Rupan Sansei: Sweet Lost Night) (2008)']
589: ['Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Lupin III: Sweet Lost Night (Rupan Sansei: Sweet Lost Night) (2008)', 'Osmosis Jones (2001)']
590: ['Rubber (2010)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Lupin III: Sweet Lost Night (Rupan Sansei: Sweet Lost Night) (2008)']
591: ['Rubber (2010)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Lupin III: Sweet Lost Night (Rupan Sansei: Sweet Lost Night) (2008)']
592: ['Rubber (2010)', 'Lupin III: First Contact (Rupan Sanse

In [27]:
# the final filter is based upon ranking by genre and rating of the movies 
# (to recommend the most appropriate movies by genre among top ranking movies)
# multiply genres by rating and normalize user rating vector to sum to 1
ratings = df.groupby('title')[['rating']].mean().sort_index(level=0, ascending=True, axis=0).T
users_ratings_new1=users_ratings_new*ratings
users_ratings_new1 = users_ratings_new1 / tf.reduce_sum(users_ratings_new1, axis=1, keepdims=True)
users_ratings_new1

<tf.Tensor: shape=(668, 10316), dtype=float64, numpy=
array([[2.45201303e-04, 1.59467241e-04, 5.56061892e-05, ...,
        1.23123502e-04, 3.97490005e-05, 3.97725624e-05],
       [1.99856168e-04, 1.72623020e-04, 6.14942056e-05, ...,
        8.64370084e-05, 5.16057177e-05, 6.19334499e-05],
       [1.71268050e-04, 1.21084190e-04, 6.87572463e-05, ...,
        6.21494071e-05, 7.69344474e-05, 7.60794466e-05],
       ...,
       [1.87214210e-04, 1.19252094e-04, 7.19757785e-05, ...,
        6.90702837e-05, 5.99907501e-05, 5.76578086e-05],
       [1.67850941e-04, 1.42344388e-04, 6.38724819e-05, ...,
        6.03075061e-05, 6.80167661e-05, 6.97716679e-05],
       [4.90804100e-04, 2.31170814e-04, 0.00000000e+00, ...,
        1.54282880e-04, 0.00000000e+00, 0.00000000e+00]])>

In [28]:
# printing top 3 rated movies for each user
top=3
top_movies1 = tf.nn.top_k(users_ratings_new1, top)[1]
top_movies1

<tf.Tensor: shape=(668, 3), dtype=int32, numpy=
array([[1764, 4658, 5617],
       [4658, 1764, 5617],
       [5617, 4658, 1764],
       ...,
       [4658, 5617, 2203],
       [4658, 5617, 1764],
       [4658, 5617, 2203]])>

In [29]:
# printing recommended movies for each user
for i in range(len(users_list)):
    movie_names = [movies_list[index] for index in top_movies1[i]]
    print('{}: {}'.format(users_list[i], movie_names))

1: ['Chase, The (1994)', 'Interstate 60 (2002)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)']
2: ['Interstate 60 (2002)', 'Chase, The (1994)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)']
3: ['Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Interstate 60 (2002)', 'Chase, The (1994)']
4: ['Interstate 60 (2002)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Saddest Music in the World, The (2003)']
5: ['Feast (2014)', 'Boxtrolls, The (2014)', 'Toy Story 3 (2010)']
6: ['Interstate 60 (2002)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Chase, The (1994)']
7: ['Chase, The (1994)', 'Interstate 60 (2002)', 'Sherlock Holmes: A Game of Shadows (2011)']
8: ['Interstate 60 (2002)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Saddest Music in the World, The (2003)']
9: ['Chase, The (1994)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (200

210: ['Interstate 60 (2002)', 'Chase, The (1994)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)']
211: ['Saddest Music in the World, The (2003)', 'Love Exposure (Ai No Mukidashi) (2008)', 'Feast (2014)']
212: ['Love Exposure (Ai No Mukidashi) (2008)', 'Saddest Music in the World, The (2003)', 'Edukators, The (Die Fetten Jahre sind vorbei) (2004)']
213: ['Interstate 60 (2002)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Chase, The (1994)']
214: ['Interstate 60 (2002)', 'Feast (2014)', 'Saddest Music in the World, The (2003)']
215: ['Batman/Superman Movie, The (1998)', 'Green Lantern: First Flight (2009)', 'Interstate 60 (2002)']
216: ['Chase, The (1994)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Interstate 60 (2002)']
217: ['Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Gentlemen of Fortune (Dzhentlmeny udachi) (1972)', 'Interstate 60 (2002)']
218: ['Interstate 60 (2002)', 'Lupin III: Fi

395: ['Interstate 60 (2002)', 'Cul-de-sac (1966)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)']
396: ['Interstate 60 (2002)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Inception (2010)']
397: ['Interstate 60 (2002)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Cul-de-sac (1966)']
398: ['Chase, The (1994)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Inception (2010)']
399: ['Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Interstate 60 (2002)', 'Cul-de-sac (1966)']
400: ['Interstate 60 (2002)', 'Chase, The (1994)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)']
401: ['Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Cul-de-sac (1966)', 'Interstate 60 (2002)']
402: ['Chase, The (1994)', 'Interstate 60 (2002)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)']
403: ['Interstate 60 (2002)', 'Inception (201

598: ['Interstate 60 (2002)', 'Inception (2010)', 'Tetsuo, the Ironman (Tetsuo) (1988)']
599: ['Interstate 60 (2002)', 'Chase, The (1994)', 'Tetsuo, the Ironman (Tetsuo) (1988)']
600: ['Interstate 60 (2002)', 'Saddest Music in the World, The (2003)', 'Feast (2014)']
601: ['Interstate 60 (2002)', 'Cul-de-sac (1966)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)']
602: ['Interstate 60 (2002)', 'Chase, The (1994)', 'Children of Men (2006)']
603: ['Interstate 60 (2002)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Cul-de-sac (1966)']
604: ['Interstate 60 (2002)', 'Inception (2010)', 'Chase, The (1994)']
605: ['Interstate 60 (2002)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Love Exposure (Ai No Mukidashi) (2008)']
606: ['Interstate 60 (2002)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontakuto) (2002)', 'Watchmen (2009)']
607: ['Interstate 60 (2002)', 'Lupin III: First Contact (Rupan Sansei: Faasuto Kontaku